In [1]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('soft/')

from qick import *
from qick_training import *
from qick.averager_program import QickSweep
from qick.averager_program import merge_sweeps

import numpy as np
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt

In [2]:
# Load bitstream with custom overlay
soc = QickSoc('./qick_training.bit')
soccfg = soc

print(soccfg)


QICK configuration:

	Board: ZCU216

	Global clocks (MHz): tProcessor 349.997, RF reference 245.760

	6 signal generator channels:
	0:	axis_sg_int4_v1 - tProc output 0, envelope memory 4096 samples
		DAC tile 2, blk 0, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	1:	axis_sg_int4_v1 - tProc output 1, envelope memory 4096 samples
		DAC tile 2, blk 1, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	2:	axis_sg_int4_v1 - tProc output 2, envelope memory 4096 samples
		DAC tile 2, blk 2, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	3:	axis_sg_int4_v1 - tProc output 3, envelope memory 4096 samples
		DAC tile 2, blk 3, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz
	4:	axis_signal_gen_v6 - tProc output 4, envelope memory 65536 samples
		DAC tile 3, blk 0, 32-bit DDS, fabric=307.200 MHz, f_dds=4915.200 MHz
	5:	axis_sg_mux4_v1 - tProc output 6, envelope memory 0 samples
		DAC tile 0, blk 0, 16-bit DDS, fabric=307.200 MHz, f_dds=1228.800 MHz

	8 readout channels:
	0:	axis_pf

In [61]:
class GainSweep(NDAveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"][0], nqz=1)
        self.declare_gen(ch=self.cfg["gen_ch"][1], nqz=1)
        
        # First channel.
        freq0 = self.freq2reg(self.cfg['pulse_freq'][0], gen_ch=self.cfg['gen_ch'][0])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][0], 
                                     style  = self.cfg['style'], 
                                     freq   = freq0,
                                     gain   = self.cfg['pulse_gain'][0], 
                                     length = self.us2cycles(self.cfg['pulse_length'][0],gen_ch=self.cfg['gen_ch'][0]))
        
        # Second channel.
        freq1 = self.freq2reg(self.cfg['pulse_freq'][1], gen_ch=self.cfg['gen_ch'][1])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][1], 
                                     style  = self.cfg['style'], 
                                     freq   = freq1,
                                     gain   = self.cfg['pulse_gain'][1], 
                                     length = self.us2cycles(self.cfg['pulse_length'][1],gen_ch=self.cfg['gen_ch'][1]))        
        
        # Sweep defination
        self.res_r_gain0 = self.get_gen_reg(self.cfg['gen_ch'][0], 'gain')
        self.res_r_gain1 = self.get_gen_reg(self.cfg['gen_ch'][1], 'gain')

        # Add gain sweep.
        gstart = int(self.cfg['pulse_gain'][0] * self.cfg['g_start'])
        gstop  = int(self.cfg['pulse_gain'][0] * self.cfg['g_stop'])
        gain0_sweep = QickSweep(self, self.res_r_gain0, gstart, gstop, self.cfg["g_expts"])
        
        gstart = int(self.cfg['pulse_gain'][1] * self.cfg['g_stop'])
        gstop  = int(self.cfg['pulse_gain'][1] * self.cfg['g_start'])
        gain1_sweep = QickSweep(self, self.res_r_gain1, gstart, gstop, self.cfg["g_expts"])
        
#         self.add_sweep(merge_sweeps([gain0_sweep,gain1_sweep]))
        self.add_sweep(gain1_sweep)


        self.synci(200)  # give processor some time to configure pulses

    def body(self):
        # Registers.
        self.set_pulse_registers(ch=self.cfg['gen_ch'][0], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][0],self.cfg['gen_ch'][0]),
                                 phrst=self.cfg['phrst'][0], 
                                 mode="oneshot")
        self.set_pulse_registers(ch=self.cfg['gen_ch'][1], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][1],self.cfg['gen_ch'][1]),
                                 phrst=self.cfg['phrst'][1], 
                                 mode="oneshot")
        
        # Trigger.
        self.trigger(pins=[0],t=40)
        
        # FIX: sweep not good for interpolated sg.
        self.regwi(2,13,0)
        self.bitwi(2,13,14, "<<", 16)

        # Pulses.
        self.pulse(ch     = self.cfg['gen_ch'][0],
                   t      = self.us2cycles(self.cfg['pulse_start'][0], gen_ch = self.cfg['gen_ch'][0]))
        
        self.pulse(ch     = self.cfg['gen_ch'][1],
                   t      = self.us2cycles(self.cfg['pulse_start'][1], gen_ch = self.cfg['gen_ch'][1]))                
        
        
        # Period.
        self.synci(self.us2cycles(self.cfg['period']))

In [62]:
config={"gen_ch"      : [1,3],
        "reps"        : 200000000,
        "style"       : 'const',
        "pulse_length": [2,3],
        "pulse_start" : [0,0],
        "pulse_gain"  : [30000,30000], # [DAC units]
        "pulse_freq"  : [3,3], # [MHz]
        "period"      : 8,
        "phase"       : [0,0],
        "phrst"       : [1,1]
       }

expt_cfg = {
    "g_start": 0.2,  # [DAC units]
    "g_stop": 0.9,  # [DAC units]
    "g_expts": 5,
}

config.update(**expt_cfg)

prog = GainSweep(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()

In [17]:
print(prog)


// Program

                          regwi 1, $13, 491520000;//gain = 30000 | addr = 0
                          bitwi 1, $13, $13 << 2;
                          regwi 2, $13, 491520000;//gain = 30000 | addr = 0
                          bitwi 2, $13, $13 << 2;
                          synci 200;
                          regwi 0, $13, 0;
                          regwi 0, $14, 199999999;
LOOP_rep:                 regwi 1, $14, 6000;   //'gen1_gain' <= 6000 
                          regwi 2, $14, 27000;  //'gen3_gain' <= 27000 
                          regwi 0, $15, 4;
LOOP_gen1_gain-gen3_gain: regwi 1, $11, 160;    //phase = 0 | freq = 160
                          regwi 1, $15, 1639014;//stdysel | mode | outsel = 0b11001 | length = 614 
                          regwi 2, $11, 160;    //phase = 0 | freq = 160
                          regwi 2, $15, 1639322;//stdysel | mode | outsel = 0b11001 | length = 922 
                          regwi 0, $31, 1;      //out = 0b00000000000000

In [65]:
class GainFreqSweep(NDAveragerProgram):
    def initialize(self):
        # Set the nyquist zone
        self.declare_gen(ch=self.cfg["gen_ch"][0], nqz=1)
        self.declare_gen(ch=self.cfg["gen_ch"][1], nqz=1)
        
        # First channel.
        freq = self.freq2reg(self.cfg['pulse_freq'][0], gen_ch=self.cfg['gen_ch'][0])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][0], 
                                     style  = self.cfg['style'], 
                                     freq   = freq,
                                     gain   = self.cfg['pulse_gain'][0], 
                                     length = self.us2cycles(self.cfg['pulse_length'][0],gen_ch=self.cfg['gen_ch'][0]))
        
        # Second channel.
        freq = self.freq2reg(self.cfg['pulse_freq'][1], gen_ch=self.cfg['gen_ch'][1])
        self.default_pulse_registers(ch     = self.cfg['gen_ch'][1], 
                                     style  = self.cfg['style'], 
                                     freq   = freq,
                                     gain   = self.cfg['pulse_gain'][1], 
                                     length = self.us2cycles(self.cfg['pulse_length'][1],gen_ch=self.cfg['gen_ch'][1]))        
        
        # Sweep defination        
        self.res_r_gain = self.get_gen_reg(self.cfg['gen_ch'][0], 'gain')
        self.res_r_freq = self.get_gen_reg(self.cfg['gen_ch'][1], 'freq')
        
        # Add freq sweep.
        self.add_sweep(QickSweep(self, self.res_r_freq, self.cfg['f_start'], self.cfg['f_stop'], self.cfg["f_expts"]))

#         # Add gain sweep.
#         gstart = int(self.cfg['pulse_gain'][0] * self.cfg['g_start'])
#         gstop  = int(self.cfg['pulse_gain'][0] * self.cfg['g_stop'])
#         self.add_sweep(QickSweep(self, self.res_r_gain, gstart, gstop, self.cfg["g_expts"]))
        
#         self.add_sweep(merge_sweeps([freq0_sweep,gain1_sweep]))

        self.synci(200)  # give processor some time to configure pulses

    def body(self):
        # Registers.
        self.set_pulse_registers(ch=self.cfg['gen_ch'][0], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][0],self.cfg['gen_ch'][0]),
                                 phrst=self.cfg['phrst'][0], 
                                 mode="oneshot")
        self.set_pulse_registers(ch=self.cfg['gen_ch'][1], 
                                 phase=soccfg.deg2reg(self.cfg['phase'][1],self.cfg['gen_ch'][1]),
                                 phrst=self.cfg['phrst'][1], 
                                 mode="oneshot")
        # Trigger.
        self.trigger(pins=[0],t=40)
        
        # FIX: sweep not good for interpolated sg.
        self.regwi(2,13,0)
        self.bitwi(2,13,14, "<<", 16)        

        # Pulses.
        self.pulse(ch     = self.cfg['gen_ch'][0],
                   t      = self.us2cycles(self.cfg['pulse_start'][0], gen_ch = self.cfg['gen_ch'][0]))
        
        self.pulse(ch     = self.cfg['gen_ch'][1],
                   t      = self.us2cycles(self.cfg['pulse_start'][1], gen_ch = self.cfg['gen_ch'][1]))                 
        
        
        # Period.
        self.synci(self.us2cycles(self.cfg['period']))

In [68]:
config={"gen_ch"      : [1,3],
        "reps"        : 200000000,
        "style"       : 'const',
        "pulse_length": [2,3],
        "pulse_start" : [0,0],
        "pulse_gain"  : [30000,30000], # [DAC units]
        "pulse_freq"  : [3,3], # [MHz]
        "period"      : 8,
        "phase"       : [0,0],
        "phrst"       : [1,1]
       }
expt_cfg = {
    # Gain Sweep.
    "g_start": 0.2,
    "g_stop" : 0.9,
    "g_expts": 5,
    
    # Frequency Sweep.
    "f_start" : 3,
    "f_stop"  : 20,
    "f_expts" : 3
}

config.update(**expt_cfg)

prog = GainFreqSweep(soccfg, config)
prog.config_all(soccfg)
soc.tproc.start()

In [67]:
print(prog)


// Program

                regwi 1, $13, 491520000;        //gain = 30000 | addr = 0
                bitwi 1, $13, $13 << 2;
                regwi 2, $13, 491520000;        //gain = 30000 | addr = 0
                bitwi 2, $13, $13 << 2;
                synci 200;
                regwi 0, $13, 0;
                regwi 0, $14, 199999999;
LOOP_rep:       regwi 2, $11, 267;              //'gen3_freq' <= 267 (5 MHz)
                regwi 0, $15, 2;
LOOP_gen3_freq: regwi 1, $11, 160;              //phase = 0 | freq = 160
                regwi 1, $15, 1639014;          //stdysel | mode | outsel = 0b11001 | length = 614 
                regwi 2, $11, 160;              //phase = 0 | freq = 160
                regwi 2, $15, 1639322;          //stdysel | mode | outsel = 0b11001 | length = 922 
                regwi 0, $31, 1;                //out = 0b0000000000000001
                seti 7, 0, $31, 40;             //ch =0 out = $31 @t = 40
                seti 7, 0, $0, 50;              //ch 